In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

# Load Raw Data

In [2]:
medal_standing_data = pd.read_excel("raw_data\medal_standings.xlsx", sheet_name="Data")
medal_standing_data.head()

,games_code,games_year,games_city,games_country,games_continent,games_start,games_end,games_season,npc,npc_new,npc_name,rank_type,npc_rank,npc_gold,npc_silver,npc_bronze,sport_code,sport
0,PG1972,1972,Heidelberg,West Germany,Europe[a],2 August,11 August,Summer,FRG,GER,Germany,Overall,1.0,28,17,22,NaN,NaN
1,PG1972,1972,Heidelberg,West Germany,Europe[a],2 August,11 August,Summer,USA,USA,United States,Overall,2.0,17,27,31,NaN,NaN
2,PG1972,1972,Heidelberg,West Germany,Europe[a],2 August,11 August,Summer,GBR,GBR,United Kingdom,Overall,3.0,16,15,21,NaN,NaN
3,PG1972,1972,Heidelberg,West Germany,Europe[a],2 August,11 August,Summer,RSA,RSA,South Africa,Overall,4.0,16,12,13,NaN,NaN
4,PG1972,1972,Heidelberg,West Germany,Europe[a],2 August,11 August,Summer,NED,NED,Netherlands,Overall,5.0,14,13,11,NaN,NaN


In [5]:
medal_standing_dict = pd.read_excel(
    "raw_data\medal_standings.xlsx", sheet_name="Data Dictionary", header = None, names = ["column_name", "explanation"]
)
medal_standing_dict

,column_name,explanation
0,games_code,The year and type of games. For years with bot...
1,games_year,The year the games were held
2,games_city,Host city
3,games_country,Host country
4,games_continent,Continent of host country
5,games_start,Date the games started
6,games_end,Date the games ended
7,games_season,Summer or winter games
8,npc,"Country code (IOC code) original to the data, ..."
9,npc_new,Copy of npc but codes have been updated to ref...


# EDA

In [ ]:
medal_standing_data